In [1]:
import pickle
import nltk
import string
import numpy as np
import pandas as pd
import regex as re
import tensorflow as tf
from nltk.corpus import stopwords, words
from nltk.tokenize import word_tokenize,wordpunct_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem import PorterStemmer
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences
import time

pd.set_option('display.max_colwidth', 140)

Using TensorFlow backend.


In [2]:
words = words.words()
newWords = ['url','retweet','usermention','hashtag']
words.extend(newWords)
words = set(words)
stop_words = set(stopwords.words("english"))
wn = WordNetLemmatizer()

In [22]:
ps =PorterStemmer()
start1 = time.time()
a = ps.stem("carries")
b = ps.stem("carried")
c = ps.stem("carrying")
end1 = time.time()
print("url :", a) 
print("retweet :", b) 
print("usermention :", c)
print("Time Taken: ", end1-start1)

url : carri
retweet : carri
usermention : carri
Time Taken:  0.0009989738464355469


In [24]:
start2 = time.time()
d = wn.lemmatize("carries")
e = wn.lemmatize("carried")
f = wn.lemmatize("carrying")
end2 = time.time()
print("url :", d) 
print("retweet :", e) 
print("usermention :", f) 
print("Time Taken: ", end2-start2)

url : carry
retweet : carried
usermention : carrying
Time Taken:  0.0


In [3]:
"hashtag" in words

True

In [4]:
print("url :", wn.lemmatize("url", pos='v')) 
print("retweet :", wn.lemmatize("retweet", pos='v')) 
print("usermention :", wn.lemmatize("usermention", pos='v')) 
print("hashtag :", wn.lemmatize("hashtag", pos='v')) 

url : url
retweet : retweet
usermention : usermention
hashtag : hashtag


In [5]:
f1 = open("dataset/legitimate_users_tweets.txt", errors = 'ignore')
input_file1 = f1.read()

In [6]:
f2 = open("dataset/content_polluters_tweets.txt", errors = 'ignore')
input_file2 = f2.read()

In [7]:
def text_to_df(file):
    parsedData = file.replace('\t', '\n').split('\n')
    textList = parsedData[2::4]
    df = pd.DataFrame({'Text': textList})
    return df

In [8]:
tweets_ham = text_to_df(input_file1)
tweets_ham.head()

,Text
0,â€¦ at house party in Daybreak. Not as weird as I'd feared. ;)
1,Taxiing â€” at SLC Salt Lake City International http://gowal.la/s/b7V
2,Almost home! â€” at PDX Portland International http://gowal.la/s/66X
3,Lunch! â€” at Ten01 http://gowal.la/s/Awa
4,Mm â€¦ books â€” at @Powells http://gowal.la/s/6fe


In [9]:
tweets_ham.describe()

,Text
count,3259693
unique,3202667
top,@tarot3
freq,255


In [10]:
tweets_spam = text_to_df(input_file2)
tweets_spam.head()

,Text
0,MELBOURNE ENQUIRY: Seeking a variety of acts for our end of year show. Payment is $120 per slot or $200 for 2.... http://bit.ly/4Ah3fF
1,THE BURLESQUE BOOTCAMP SYDNEY - Open Date tickets now available from http://bbootcampsyd.eventbrite.com/ for Jan /... http://bit.ly/rCenZ
2,THE BURLESQUE BOOTCAMP SYDNEY - Open Date tickets now available from http://bbootcampsyd.eventbrite.com/ for Jan /... http://bit.ly/1v5hvb
3,THE BURLESQUE BOOTCAMP SYDNEY - Open Date tickets now available from http://bbootcampsyd.eventbrite.com/ for Jan /... http://fb.me/3rzipF0
4,"Come to ""The Burlesque Bootcamp - Sydney"" Saturday, 23 January 2010 at 10:00 until <br />Sunday, 24 January 2010 at... http://bit.ly/38simD"


In [11]:
tweets_spam.describe()

,Text
count,2353473
unique,1967975
top,ALL NEW TWITTER AUTOMATION SOFTWARE FOR YOUR PC http://bit.ly/6BiLk3 WITH UNLIMITED ACCOUNTS AND FUN TO USE - CHECKOUT OUR VIDEO
freq,586


In [12]:
def clean_data(trial_text):
    for i in trial_text.index:
        trial_text.at[i,"Text"] = re.sub(r"http\S+", "url", trial_text.at[i,"Text"])
        trial_text.at[i,"Text"] = re.sub(r"@\S+", "usermention", trial_text.at[i,"Text"])
    trial_text.drop_duplicates(subset='Text', keep='first', inplace=True)
    for i in trial_text.index:
        trial_text.at[i,"Text"] = re.sub(r"#", "", trial_text.at[i,"Text"])
        trial_text.at[i,"Text"] = re.sub(r"\bRT\b", "retweet", trial_text.at[i,"Text"])
        trial_text.at[i,"Text"] = ' '.join(word for word in wordpunct_tokenize(trial_text.at[i,"Text"]) 
                                           if word.lower() not in stop_words)
        trial_text.at[i,"Text"] = ' '.join(word.strip(string.punctuation) for word in trial_text.at[i,"Text"].split())
        trial_text.at[i,"Text"] = wordpunct_tokenize(trial_text.at[i,"Text"].lower())
        trial_text.at[i,"Text"] = ' '.join([wn.lemmatize(word, pos='v') for word in trial_text.at[i,"Text"]])
        trial_text.at[i,"Text"] = ' '.join(word for word in word_tokenize(trial_text.at[i,"Text"]) if word in words)
        trial_text.at[i,"Text"] = " ".join(word for word in word_tokenize(trial_text.at[i,"Text"]) 
                                           if not (word.isalpha() and len(word)<3))
    #Additional loop for processing glove dataset
#      for i in trial_text.index:
#          trial_text.at[i,"Text"] = re.sub(r"\burl\b", "<url>", trial_text.at[i,"Text"])
#          trial_text.at[i,"Text"] = re.sub(r"\busermention\b", "<user>", trial_text.at[i,"Text"]) 
#          trial_text.at[i,"Text"] = re.sub(r"\bhashtag\b", "<hashtag>", trial_text.at[i,"Text"])
#          trial_text.at[i,"Text"] = re.sub(r"\bretweet\b", "rt", trial_text.at[i,"Text"])
    trial_text['num_words']=trial_text["Text"].str.split().str.len()
    trial_text = trial_text.drop(trial_text[trial_text.num_words < 2].index)
    trial_text = trial_text.drop(['num_words'], axis=1)
    return trial_text

In [13]:
clean_ham_df = clean_data(tweets_ham)
clean_ham_df.head()

,Text
0,house party daybreak weird fear
1,taxi salt lake city international url
2,almost home international url
3,lunch url
4,book usermention url


In [14]:
clean_ham_df.isnull().sum()

Text    0
dtype: int64

In [15]:
clean_ham_df.describe()

,Text
count,2254052
unique,1788328
top,retweet usermention
freq,16340


In [16]:
clean_spam_df = clean_data(tweets_spam)
clean_spam_df.head()

,Text
0,enquiry seek variety act end year show payment per slot url
1,burlesque open date ticket available url url
4,come burlesque url
5,century write perform black flamingo berlin url
6,burlesque come burlesque head url


In [17]:
clean_spam_df.isnull().sum()

Text    0
dtype: int64

In [18]:
clean_spam_df.describe()

,Text
count,1688086
unique,1480393
top,subscribe channel url
freq,2638


In [19]:
# Check length of longest tweet
x = clean_ham_df["Text"].str.split(" ")
max_len1 = 0
max_len2 = 0
max_len3 = 0
max_row1 = 0
count = 0
index = 0

for i in x:
    if len(i)>max_len1:
        max_len3 = max_len2
        max_len2 = max_len1
        max_len1 = len(i)
        max_row2 = max_row1
        max_row1 = i
        index = count
    count = count+1
        
print(max_len1)
print(max_len2)
print(max_len3)
print(index)
print(max_row1)
print(max_row2)

35
17
16
22114
['sad', 'sad', 'sad', 'sad', 'sad', 'sad', 'sad', 'sad', 'sad', 'sad', 'sad', 'sad', 'sad', 'sad', 'sad', 'sad', 'sad', 'sad', 'sad', 'sad', 'sad', 'sad', 'sad', 'sad', 'sad', 'sad', 'sad', 'sad', 'sad', 'sad', 'sad', 'sad', 'sad', 'sad', 'sad']
['need', 'supplemental', 'gear', 'bag', 'leg', 'bag', 'butt', 'pack', 'leg', 'bag', 'smaller', 'butt', 'pack', 'flop', 'around', 'double', 'time']


In [29]:
num = 0
for i in x:
    if len(i)==21:
        print(i)
        num+=1
print("number of length {} tweets: {}".format(len(i),num))

['rt', '<user>', 'want', 'bother', 'rt', '<user>', 'rt', '<user>', 'rt', '<user>', 'rt', '<user>', 'rt', '<user>', 'rt', '<user>', 'rt', '<user>', 'rt', '<user>', 'rt']
['ring', 'ring', 'ring', 'ring', 'ring', 'ring', 'ring', 'ring', 'ring', 'ring', 'ring', 'ring', 'ring', 'ring', 'ring', 'ring', 'ring', 'ring', 'ring', 'ring', '<url>']
['rt', '<user>', '<user>', '<hashtag>', '<hashtag>', '<hashtag>', '<hashtag>', '<hashtag>', '<hashtag>', '<hashtag>', '<hashtag>', '<hashtag>', '<hashtag>', '<hashtag>', '<hashtag>', '<hashtag>', '<hashtag>', '<hashtag>', '<hashtag>', '<hashtag>', '<hashtag>']
['half', 'past', 'seven', 'half', 'past', 'seven', 'half', 'past', 'seven', 'half', 'past', 'seven', 'half', 'past', 'seven', 'half', 'past', 'seven', 'like', 'ending', 'horror']
['let', 'get', 'ready', '<hashtag>', '<hashtag>', '<hashtag>', '<hashtag>', '<hashtag>', '<hashtag>', '<hashtag>', '<hashtag>', '<hashtag>', '<hashtag>', '<hashtag>', '<hashtag>', '<hashtag>', '<hashtag>', '<hashtag>', '<

In [20]:
clean_ham_df['Label'] = 'Ham'
clean_ham_df.describe()

,Text,Label
count,2250118,2250118
unique,1794583,1
top,retweet usermention,Ham
freq,16349,2250118


In [21]:
clean_spam_df['Label'] = 'Spam'
clean_spam_df.describe()

,Text,Label
count,1688741,1688741
unique,1489713,1
top,channel url,Spam
freq,2662,1688741


In [22]:
tweets_text = clean_ham_df.append(clean_spam_df, ignore_index=True)
tweets_text.describe()

,Text,Label
count,3938859,3938859
unique,3246550,2
top,retweet usermention,Ham
freq,17306,2250118


In [23]:
#Summary of Tweet Text dataset
print("Out of {} rows, {} are spams, {} are non-spams".format(len(tweets_text),
                                                              len(tweets_text[tweets_text['Label']=='Spam']),
                                                              len(tweets_text[tweets_text['Label']=='Ham'])))

Out of 3938859 rows, 1688741 are spams, 2250118 are non-spams


In [24]:
tweets_text.isnull().sum()

Text     0
Label    0
dtype: int64

In [25]:
#export cleaned Tweet Text dataset
tweets_text.to_csv('dataset_improved_260321.txt', sep='\t', index=False, header=False)